In [2]:
%pip install accelerate bitsandbytes transformers peft datasets scipy beartype einops Jinja2 numpy pytorch-custom-utils torchtyping tqdm x-transformers sentencepiece


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.0.11/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --pre torch --index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://download.pytorch.org/whl/nightly/cpu

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.0.11/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import sys 
sys.path.append("./self-rewarding-lm-pytorch/")

import torch
from torch import Tensor
from self_rewarding_lm_pytorch import SelfRewardingTrainer
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel


In [6]:
# TODO: Replace with SFT dataset + prompts here
from typing import Type, Any
from torch.utils.data import Dataset

def create_mock_dataset(
    length: int,
    output: Any
) -> Type[Dataset]:

    class MockDataset(Dataset):
        def __len__(self):
            return length

        def __getitem__(self, idx):
            if callable(output):
                return output()

            return output

    return MockDataset

SFTDataset = create_mock_dataset(100, lambda: (torch.randint(0, 256, (256,)), torch.tensor(1)))
PromptDataset = create_mock_dataset(100, lambda: 'mock prompt')
next(iter(SFTDataset()))

(tensor([135, 113,  62,  27, 211,  21,  73, 128, 168, 210,  37, 176,  27,   8,
         115,  55,  23,   3, 125,  77,  55, 226, 106,  62, 102, 148, 193, 132,
         167, 170,  64,  74, 205, 144, 241, 135,  70, 176,  65, 219,   0, 189,
          72,  96, 173, 171,  25,  64, 101, 131, 237, 127, 173, 213, 149,  68,
         229, 232,  89, 220, 239, 237,  84, 191, 217, 186,  65, 209, 113, 174,
          31, 109,  61, 225, 137, 156,  14, 144, 192, 203,  46, 101, 232, 148,
         229, 243, 212,  56,  87, 161,  36, 218, 192, 142, 200,  99, 240, 155,
         161, 161, 177, 211, 253,  71,  65, 253, 149, 196,  48,  49, 217, 176,
         197, 197, 182, 126,   5,  81, 170,  44,  91,  23, 168, 129,  89,  64,
          77, 185, 253, 169, 215, 194, 129,  59, 207,  47, 225,  51, 109, 210,
         126,  74,  39,  36, 114, 183,  59,  76,  79, 174,  77,  75,  44, 226,
         213, 220, 248,  59, 242, 120,  75,  83,   3, 100, 127,  62, 145, 223,
         116, 136,   9,  22,  10, 220,  70, 150,  85

In [7]:
# TODO: mixtral tokenizer encoder and decoder here

def decode_tokens(tokens: Tensor) -> str:
    decode_token = lambda token: str(chr(max(32, token)))
    return ''.join(list(map(decode_token, tokens)))

def encode_str(seq_str: str) -> Tensor:
    return Tensor(list(map(ord, seq_str)))

decode_tokens(next(iter(SFTDataset()[0]))), encode_str("hello there")

('\x86ï  "Ë\x9d\x83GQ\x83\x91w2Âh\x8e#£å\x97[ÕôkOî\x9c Ç ÙÔ~\x80 \x94Þ«"\\Ýp ¢yä )Fñþå °õÞ\x99(OAXä¿ûÍÕ³\x90øÊ @&3ß Ê{¹¨ñ£ µâ¢U¿ÆÛ\x936*¦æÐ \x86  `Gç\x97\x8f¿Ùd\x9bÚ q÷ x\x9f\x8856^ i¯?I¶# ÿ;WõÙ\x7f\x99C¯+=åùt  \x8e3ÎÕ×GëNÌ²[;¾eµ\x81Ñ\\ù(z\x80 © Ã³À\x85 ÿIíêK¾\x89ßw½º eÖù=f¿÷\x959tY#ûPÞO\x8f¨¾{<\x8f\x85\xa0ö¹ä &b,\x85 \x86Ü\x94 »¹è+¼QétkC@\x93¾¨Ï"y¢Nüi OÐþBÂcDü\x96ç',
 tensor([104., 101., 108., 108., 111.,  32., 116., 104., 101., 114., 101.]))

In [8]:
model_name = "teknium/OpenHermes-2.5-Mistral-7B"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
tokenizer.eos_token, tokenizer.bos_token, tokenizer.pad_token

('<|im_end|>', '<s>', '<|im_end|>')

In [10]:
def chatml_format(example, system=""):
    # Format system
    if len(system) > 0:
        message = {"role": "system", "content": system}
        system = tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system = ""
    # Format instruction, add assistant start to end of input
    message = {"role": "user", "content": example['user']}
    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)

    response = example['assistant'] + "<|im_end|>\n"

    return {
        "input": system + prompt,
        "label": response
    }

In [13]:
from data.prompts import fields, system
print("Fields system prompt:", len(tokenizer(fields)['input_ids']))
print("System prompt len:", len(tokenizer(system)['input_ids']) + len(tokenizer(fields)['input_ids']) )

Fields system prompt: 443
System prompt len: 2785


In [14]:
import json

with open('data/results/FINAL_samples.json', 'r') as f:
    samples = json.load(f)

formatted_samples = [chatml_format(sample, system=system) for sample in samples]
formatted_samples[0]


{'input': '<|im_start|>system\nYou are a helpful assisant designed to interact with the Google Places API based on user queries. Your job is to output the correct function call and parameters to be executed based on the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\nThe JSON output should be preceded with the token "<|api_call|>" and end with the token "<|api_call_end|>". For example: "<|api_call|>{...}<|api_call_end|>".\n\nHere is the output schema:\n{\n    "properties": {\n        "function": {\n            "title": "Function Name",\n            "description": "The name of the function to be called",\n 

In [15]:
# max_length length of max sample in data
max_length = 420
message = {"role": "user", "content": "Do something active in Central Park."}
prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
print(prompt)
tokenizer.encode(prompt,
                truncation=True,
                max_length=max_length,
                padding="max_length",
                )  

<|im_start|>user
Do something active in Central Park.<|im_end|>
<|im_start|>assistant



[32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,
 32000,


In [39]:
# TODO: initizlize mixtral qlora w/ bandb (see links saved)
# TODO: investigate ideal lora config for mixtral
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config,  # loading model in 4-bit
    device_map='auto', # to use max gpu resources if exist
)

In [40]:
# TODO: investigate hyperparams for training
# investigate kwargs for SelfRewardingTrainer (pre pairs, spin, accelerate)
# investigate ideal dpo trainer kwargs
# experiment with llm as judge prompt
# KTO, pair sampling strat, kv cache?
# get to train on cpu, lower dpo iters/epochs

# NOTE: PLAN
# Need to modify self rewarding trainer / add config for calling places API to give context to LLM as judge for rewarding
# model responses
# i.e when presented with the user query and generated response, the LLM as judge needs real response from Places API
# to make judgement (i.e. "Does the following response answer the user query")
# we need function to validate and execute json param generated by LLM 
# ability to cache/store responses for newly generated responses # NOTE NOTE NOTE IMPORTANT for my wallet
# handle if request fails, i.e. say "Request failed to return any data, automatic score of 0" idk

trainer = SelfRewardingTrainer(
    model,
    train_sft_dataset = SFTDataset(),
    spin = False,
    num_preference_pairs = [1, 1],
    preference_max_seq_len = 64,
    prompt_dataset = PromptDataset(),
    tokenizer_encode = encode_str,
    tokenizer_decode = decode_tokens,
    accelerate_kwargs = dict(
        cpu = True
    ),
    dpo_trainer_kwargs = dict(
        batch_size = 1
    )
)

trainer(overwrite_checkpoints = True)

100%|██████████████████████████████████████████| 53/53 [00:00<00:00, 155.88it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████| 53/53 [00:00<00:00, 177.17it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████| 53/53 [00:00<00:00, 189.12it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████| 53/53 [00:00<00:00, 166.00it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████| 53/53 [00:00<00:00, 171.84it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████| 53/53 [00:00<00:00, 154.19it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████████████████████████████████████| 53/53 [00

KeyboardInterrupt: 